In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tensorflow.keras import preprocessing
import random
import tensorflow as tf

In [ ]:
class_names = ['BANANA' , 'APPLE' , 'POMOGRANTE', 'LEMON']

In [ ]:
camera = cv2.VideoCapture(0)
camera_height = 800
raw_frames_type_1 = []
raw_frames_type_2 = []
raw_frames_type_3 = []
raw_frames_type_4 = []

while(True):
    _,frame = camera.read()
    frame = cv2.flip(frame,1)
    
    asc = frame.shape[1]/float(frame.shape[0])
    res = int(asc*camera_height)
    frame = cv2.resize(frame , (res,camera_height))
    
    cv2.rectangle(frame, (300,175) , (650,525) , 2)
    cv2.imshow('capturing Frames', frame)
    
    key = cv2.waitKey(1)
    
    if key & 0xFF == ord("q"):
        break
    elif key & 0xFF == ord("1"):
        raw_frames_type_1.append(frame)
        print('1 key pressed - saved TYPE_1 frame')
    elif key & 0xFF == ord("2"):
        raw_frames_type_2.append(frame)
        print('1 key pressed - saved TYPE_2 frame')
    elif key & 0xFF == ord("3"):
        raw_frames_type_3.append(frame)
        print('1 key pressed - saved TYPE_3 frame')
    elif key & 0xFF == ord("4"):
        raw_frames_type_4.append(frame)
        print('1 key pressed - saved TYPE_4 frame')
        
camera.release()
cv2.destroyAllWindows()

# Crop The frames and resize it to 399*399

In [ ]:
save_height = 399
save_width = 399

In [ ]:
for i in range(1,5):
    name = 'C:/Users/HP/Desktop/image_type_{}'.format(i)
    os.makedirs(name , exist_ok= True)

In [ ]:
for i, frame in enumerate(raw_frames_type_1):
    roi = frame[75+2:425-2, 300+2:650-2]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (save_width, save_height))
    cv2.imwrite('C:/Users/HP/Desktop/image_type_1/{}.jpg'.format(i), cv2.cvtColor(roi,cv2.COLOR_BGR2RGB))

In [ ]:
for i, frame in enumerate(raw_frames_type_2):
    roi = frame[75+2:425-2, 300+2:650-2]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (save_width, save_height))
    cv2.imwrite('C:/Users/HP/Desktop/image_type_2/{}.jpg'.format(i), cv2.cvtColor(roi,cv2.COLOR_BGR2RGB))

In [ ]:
for i, frame in enumerate(raw_frames_type_3):
    roi = frame[75+2:425-2, 300+2:650-2]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (save_width, save_height))
    cv2.imwrite('C:/Users/HP/Desktop/image_type_3/{}.jpg'.format(i), cv2.cvtColor(roi,cv2.COLOR_BGR2RGB))

In [ ]:
for i, frame in enumerate(raw_frames_type_4):
    roi = frame[75+2:425-2, 300+2:650-2]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (save_width, save_height))
    cv2.imwrite('C:/Users/HP/Desktop/image_type_4/{}.jpg'.format(i), cv2.cvtColor(roi,cv2.COLOR_BGR2RGB))

# Loading The Model

In [ ]:
width = 96
height = 96

In [ ]:
def load_images(base_path):
    images = []
    path = os.path.join(base_path , '*jpg')
    for image_path in glob(path):
        image = preprocessing.image.load_img(image_path , target_size=(width,height))
        x = preprocessing.image.img_to_array(image)
        images.append(x)
        
    return images

In [ ]:
images_type_1 = load_images('C:/Users/HP/Desktop/image_type_1')
images_type_2 = load_images('C:/Users/HP/Desktop/image_type_2')
images_type_3 = load_images('C:/Users/HP/Desktop/image_type_3')
images_type_4 = load_images('C:/Users/HP/Desktop/image_type_4')

In [ ]:
X_type_1 = np.array(images_type_1)
X_type_2 = np.array(images_type_1)
X_type_3 = np.array(images_type_1)
X_type_4 = np.array(images_type_1)

X_type_1.shape

In [ ]:
X = np.concatenate((X_type_1, X_type_2, X_type_3, X_type_4), axis=0)

In [ ]:
X = X / 255.0
X.shape

In [ ]:
from tensorflow.keras.utils import to_categorical

Y_type_1 = [0 for item in enumerate(X_type_1)]
Y_type_2 = [1 for item in enumerate(X_type_2)]
Y_type_3 = [2 for item in enumerate(X_type_3)]
Y_type_4 = [3 for item in enumerate(X_type_4)]

Y = np.concatenate((Y_type_1, Y_type_2, Y_type_3, Y_type_4), axis=0)

Y = to_categorical(Y, num_classes = len(class_names))

Y.shape

# Convolution Neural Network

In [ ]:
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense 
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

conv_1 = 16
conv_1_drop = 0.2
conv_2 = 32
conv_2_drop = 0.2
dense_1_n = 1024
dense_1_drop = 0.2
dense_2_n = 512
dense_2_drop = 0.2
lr = 0.01

epochs = 30
#batch_size = 32
color_channels = 3

def build_model(conv_1_drop=conv_1_drop, conv_2_drop=conv_2_drop,
                dense_1_n=dense_1_n, dense_1_drop=dense_1_drop,
                dense_2_n=dense_2_n, dense_2_drop=dense_2_drop,
                lr=lr):
    model = Sequential()

    model.add(Convolution2D(conv_1, (3, 3),
                            input_shape=(width, height, color_channels),
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(conv_1_drop))

    model.add(Convolution2D(conv_2, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(conv_2_drop))
        
    model.add(Flatten())
        
    model.add(Dense(dense_1_n, kernel_regularizer=regularizers.l2(0.001),  activation='relu'))
    model.add(Dropout(dense_1_drop))

    model.add(Dense(dense_2_n, activation='relu'))
    model.add(Dropout(dense_2_drop))

    model.add(Dense(len(class_names), activation='sigmoid'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=lr),
                  metrics=['accuracy'])

    return model


In [ ]:
import numpy as np
np.random.seed(1)
model = build_model()
model.summary()

In [ ]:
epochs = 20
model.fit(X,Y,epochs = epochs)

In [ ]:
import time 

# get the reference to the webcam
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):
    # read a new frame
    _, frame = camera.read()
    
    # flip the frameq
    frame = cv2.flip(frame, 1)

    # rescaling camera output
    aspect = frame.shape[1] / float(frame.shape[0])
    res = int(aspect * camera_height) # landscape orientation - wide image
    frame = cv2.resize(frame, (res, camera_height))

    # add rectangle
    cv2.rectangle(frame, (300, 75), (650, 425), (240, 100, 0), 2)

    # get ROI
    roi = frame[75+2:425-2, 300+2:650-2]
    
    # parse BRG to RGB
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

    # resize
    roi = cv2.resize(roi, (width, height))
    
    # predict!
    roi_X = np.expand_dims(roi, axis=0)
    type(roi_X)
    roi_X = tf.cast(roi_X , tf.float16)
    predictions = model.predict(roi_X)
    type_1_pred, type_2_pred, type_3_pred, type_4_pred = predictions[0]

    # add text
    type_1_text = '{}: {}%'.format(class_names[0], int(type_1_pred*100))
    cv2.putText(frame, type_1_text, (70, 170), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

    # add text
    tipe_2_text = '{}: {}%'.format(class_names[1], int(type_2_pred*100))
    cv2.putText(frame, tipe_2_text, (70, 200), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

    
    # add text
    tipe_3_text = '{}: {}%'.format(class_names[2], int(type_3_pred*100))
    cv2.putText(frame, tipe_3_text, (70, 230), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

    # add text
    tipe_4_text = '{}: {}%'.format(class_names[3], int(type_4_pred*100))
    cv2.putText(frame, tipe_4_text, (70, 260), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (240, 240, 240), 2)

    # show the frame
    cv2.imshow("Test out", frame)

    key = cv2.waitKey(1)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break
    
camera.release()
cv2.destroyAllWindows()

In [ ]:
model.summary()